In [1]:
import math
import sys

sys.path.append("../ares-sc2")  # required to import sc2_helper
sys.path.append("../ares-sc2/src")  # required to import ares
sys.path.append("phantom")

import cvxpy as cp
import scipy as sp
import numpy as np

from phantom.cvxpygen.assign.assign import cpg_assign

2025-03-26 19:23:37.297 | INFO     | phantom.common.assignment:<module>:12 - cp.installed_solvers()=['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'HIGHS', 'MOSEK', 'OSQP', 'PIQP', 'PROXQP', 'SCIP', 'SCIPY', 'SCS', 'SDPA']


In [2]:
cp.installed_solvers()

['CLARABEL',
 'CVXOPT',
 'ECOS',
 'ECOS_BB',
 'GLPK',
 'GLPK_MI',
 'HIGHS',
 'MOSEK',
 'OSQP',
 'PIQP',
 'PROXQP',
 'SCIP',
 'SCIPY',
 'SCS',
 'SDPA']

In [3]:
SOLVER = cp.ECOS

In [4]:
N = 25
M = 3
max_assigned = math.ceil(N / M)
np.random.seed(0)
W = sp.stats.lognorm.rvs(s=2, size=(N, M))
B = np.full(M, max_assigned)
W

array([[3.40593534e+01, 2.22624079e+00, 7.08143073e+00],
       [8.83924358e+01, 4.18928840e+01, 1.41627379e-01],
       [6.68707685e+00, 7.38810058e-01, 8.13476937e-01],
       [2.27321926e+00, 1.33387354e+00, 1.83301458e+01],
       [4.58172448e+00, 1.27551502e+00, 2.42959959e+00],
       [1.94906283e+00, 1.98490897e+01, 6.63440220e-01],
       [1.87036837e+00, 1.81193188e-01, 6.06039905e-03],
       [3.69594839e+00, 5.63429708e+00, 2.26654139e-01],
       [9.36448324e+01, 5.45448844e-02, 1.09583544e+00],
       [6.87723989e-01, 2.14464349e+01, 1.88916030e+01],
       [1.36328176e+00, 2.13043256e+00, 1.69386618e-01],
       [1.90327721e-02, 4.98663231e-01, 1.36710852e+00],
       [1.17116182e+01, 1.10757684e+01, 4.60863380e-01],
       [5.46289895e-01, 1.22811339e-01, 5.84235700e-02],
       [3.29573697e-02, 4.94791214e+01, 3.60845870e-01],
       [4.16383485e-01, 8.16273648e-02, 4.73499519e+00],
       [3.96447918e-02, 6.53455685e-01, 1.66804448e-01],
       [2.16799981e+00, 3.60014

In [5]:
x = cp.Variable((N, M), "x")
w = cp.Parameter((N, M), name="w")
constraints = [
    cp.sum(x, 0) <= max_assigned,  # enforce even distribution
    cp.sum(x, 1) == 1,
    0 <= x,
]
problem = cp.Problem(cp.Minimize(cp.vdot(w, x)), constraints)

In [6]:
def solve_cpg():
    return cpg_assign(W, B)


solve_cpg().argmax(1)

2025-03-26 19:23:37.433 | INFO     | phantom.cvxpygen.assign.assign:load_solvers:70 - solvers=[AssignSolver(module_path='assign_12_12'), AssignSolver(module_path='assign_16_16'), AssignSolver(module_path='assign_24_24'), AssignSolver(module_path='assign_32_32'), AssignSolver(module_path='assign_4_4'), AssignSolver(module_path='assign_6_6'), AssignSolver(module_path='assign_8_8')]


array([2, 1, 2, 2, 2, 1, 2, 2, 1, 0, 2, 0, 2, 2, 1, 2, 0, 2, 2, 2, 0, 0,
       0, 2, 2])

In [7]:
def solve_cvxpy():
    w.value = W
    problem.solve(solver="ECOS")
    return x.value


solve_cvxpy().argmax(1)

array([1, 2, 1, 1, 1, 2, 2, 2, 1, 0, 2, 0, 2, 2, 0, 1, 0, 2, 0, 1, 2, 0,
       0, 0, 1])

In [8]:
A_ub = np.tile(np.identity(M), (1, N))
b_ub = np.full(M, max_assigned)

A_eq = np.repeat(np.identity(N), M, axis=1)
b_eq = np.full(N, 1.0)

c = W.flatten()

In [9]:
def solve_highs():
    return sp.optimize.linprog(
        c=c,
        A_ub=A_ub,
        b_ub=b_ub,
        A_eq=A_eq,
        b_eq=b_eq,
        method="highs",
    ).x.reshape((N, M))


solve_highs().argmax(1)

array([1, 2, 1, 1, 1, 2, 2, 2, 1, 0, 2, 0, 2, 2, 0, 1, 0, 2, 0, 1, 2, 0,
       0, 0, 1])

In [10]:
A_ub_sparse = sp.sparse.csr_matrix(A_ub)
A_eq_sparse = sp.sparse.csr_matrix(A_eq)

In [11]:
%%timeit
solve_cvxpy()

954 μs ± 143 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
%%timeit
solve_highs()

657 μs ± 67.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
%%timeit
solve_cpg()

2.8 ms ± 70.6 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
